In [1]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
from sklearn import preprocessing
import os
import seaborn as sns
from skimage.feature import hog

## Provided Features

In [2]:
features = pd.read_csv('./features/1-bg-01-090.txt','\t',header=None)
print(features.shape)
features.head()

(5646, 437)


,0,1,2,3,4,5,6,7,8,9,...,427,428,429,430,431,432,433,434,435,436
0,17,286.794434,167.822754,1.896050,0.531059,6.950572,4.000000,0.896233,0.699261,0.089623,...,0.225257,0.211130,0.260318,0.229689,0.217195,0.162070,0.194323,0.155583,0.148571,NaN
1,23,300.929199,175.344849,3.078520,0.092459,13.291861,2.828427,0.940404,0.730604,0.146226,...,0.189001,0.161111,0.199708,0.173203,0.178513,0.161606,0.132132,0.123583,0.143847,NaN
2,25,316.413452,155.776825,1.425396,3.351885,14.828127,1.000000,0.988792,0.649070,0.165094,...,0.172196,0.185987,0.162559,0.179747,0.146249,0.133324,0.208936,0.298669,0.240475,NaN
3,27,306.164948,151.834702,2.162061,0.523864,9.353746,1.000000,0.956765,0.632645,0.183962,...,0.197651,0.259199,0.222601,0.244148,0.203020,0.156068,0.205083,0.240761,0.314523,NaN
4,28,304.745514,156.542145,4.108115,1.116036,14.412641,1.000000,0.952330,0.652259,0.193396,...,0.307873,0.212699,0.193691,0.257051,0.273124,0.203735,0.231509,0.236546,0.248627,NaN


In [3]:
#last column is read as null remove it
print(features.iloc[:,-1].isnull().sum())
features.drop(features.columns[-1],axis=1,inplace=True)
features.head()

5646


,0,1,2,3,4,5,6,7,8,9,...,426,427,428,429,430,431,432,433,434,435
0,17,286.794434,167.822754,1.896050,0.531059,6.950572,4.000000,0.896233,0.699261,0.089623,...,0.163575,0.225257,0.211130,0.260318,0.229689,0.217195,0.162070,0.194323,0.155583,0.148571
1,23,300.929199,175.344849,3.078520,0.092459,13.291861,2.828427,0.940404,0.730604,0.146226,...,0.207161,0.189001,0.161111,0.199708,0.173203,0.178513,0.161606,0.132132,0.123583,0.143847
2,25,316.413452,155.776825,1.425396,3.351885,14.828127,1.000000,0.988792,0.649070,0.165094,...,0.097894,0.172196,0.185987,0.162559,0.179747,0.146249,0.133324,0.208936,0.298669,0.240475
3,27,306.164948,151.834702,2.162061,0.523864,9.353746,1.000000,0.956765,0.632645,0.183962,...,0.133310,0.197651,0.259199,0.222601,0.244148,0.203020,0.156068,0.205083,0.240761,0.314523
4,28,304.745514,156.542145,4.108115,1.116036,14.412641,1.000000,0.952330,0.652259,0.193396,...,0.230429,0.307873,0.212699,0.193691,0.257051,0.273124,0.203735,0.231509,0.236546,0.248627


- Trajectory:    2x[trajectory length] (default 30 dimension)
- HOG:           8x[spatial cells]x[spatial cells]x[temporal cells] (default 96 dimension)
- HOF:           9x[spatial cells]x[spatial cells]x[temporal cells] (default 108 dimension)
- MBHx:          8x[spatial cells]x[spatial cells]x[temporal cells] (default 96 dimension)
- MBHy:          8x[spatial cells]x[spatial cells]x[temporal cells] (default 96 dimension)

In [6]:
trajectory_range = 40
hog_range = trajectory_range+96
hof_range = hog_range+108
mbhx_range = hof_range+96
mbhy_range = mbhx_range+96

In [5]:
trajectory_features = features.iloc[:,0:trajectory_range]
hog_features = features.iloc[:,trajectory_range:hog_range]
hof_features = features.iloc[:,hog_range:hof_range]
mbhx_features = features.iloc[:,hof_range:mbhx_range]
mbhy_features = features.iloc[:,mbhx_range:mbhy_range]

## Calculating features

In [7]:
startFrame = 0
endFrame = 0
trajectoryLength = 15
samplingStride = 5
framesFeatures = 5
neighborhoodSize = 32
spatialCells = 2
temporalCells = 3

In [7]:
video = cv2.VideoCapture('./dataset/001-bg-01-090.avi')
read_frames = [video.read()[1] for x in range(trajectoryLength)]
frames = np.array(read_frames)
gray_scale = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in frames]
gray_scale = np.array(gray_scale)

In [8]:
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print( "Frames in video",length )

Frames in video 106


In [9]:
print(frames.shape)
print(gray_scale.shape)

(15, 240, 320, 3)
(15, 240, 320)


## HOG Features

In [10]:
def computeOpticalHog(image):
    features = []    
    window_size = int(neighborhoodSize/2)
    row, col = image.shape        
    for i in range(5, row, samplingStride):

        min_ind_row = i-window_size
        max_ind_row = i+window_size

        if(min_ind_row < 0):
            min_ind_row = 0
        if(max_ind_row > row):
            max_ind_row = row

        for j in range(5, col, 5):        
            min_ind_col = j-window_size       
            max_ind_col = j+window_size

            if(min_ind_col<0):
                min_ind_col=0
            if(max_ind_col>col):
                max_ind_col = col

            window = np.zeros([32,32])
            selected_window = image[min_ind_row:max_ind_row , min_ind_col:max_ind_col]
            window[:selected_window.shape[0],:selected_window.shape[1]] = selected_window
            fd = hog(window, orientations=8, pixels_per_cell=(16, 16),cells_per_block=(2, 2))
            features.append(fd)
            
    features = np.array(features)
    return features

In [11]:
def GetHogFeatures(videoFrames):
    temporal_features = []
    hog_features = []
    for i,image in enumerate(videoFrames):        
        features = computeOpticalHog(image)
        temporal_features.append(features)    
        if((i+1)%framesFeatures==0):        
            feature = np.array(temporal_features)            
            hog_features.append(feature.sum(axis=0))
            temporal_features = []
    return np.array(hog_features).reshape(-1,neighborhoodSize*temporalCells)

In [12]:
vid1_features_hog = GetHogFeatures(gray_scale)
vid1_features_hog.shape

(2961, 96)

In [13]:
pd.DataFrame(vid1_features_hog)

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,0.280069,0.086851,0.341799,0.465822,0.727716,0.195819,0.156695,0.049894,2.265499,0.013166,...,0.031193,0.010595,0.735390,0.188201,0.315501,0.239125,2.414438,0.007987,0.030322,0.461940
1,0.272107,0.091132,0.438279,0.702503,0.896280,0.188331,0.178953,0.027910,0.184156,0.060362,...,0.041963,0.002049,0.051648,0.005793,0.067280,0.669169,3.189836,0.008669,0.020224,0.000000
2,0.179091,0.058573,0.457382,0.803188,0.945531,0.120690,0.158303,0.017782,0.224960,0.090493,...,0.019977,0.000000,0.058981,0.026155,0.097789,0.910084,3.152345,0.021570,0.035500,0.006637
3,0.222636,0.089290,0.470873,0.558625,1.087925,0.315728,0.152049,0.028396,0.142159,0.023926,...,0.034988,0.006465,0.103165,0.057973,0.062684,1.842955,2.995910,0.037272,0.107933,0.020366
4,0.140473,0.023621,0.307483,0.676792,0.769711,0.033501,0.069239,0.001953,0.308679,0.080971,...,0.107463,0.020431,0.056364,0.019477,0.054093,1.348820,2.965758,0.011027,0.026639,0.001966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2956,0.441446,0.156019,0.301629,0.183472,0.578939,0.481298,0.386096,0.128000,0.297396,0.110205,...,0.041136,0.005065,0.040123,0.021091,0.088560,2.484260,2.587882,0.030735,0.025236,0.000000
2957,0.289565,0.107305,0.244658,0.164613,0.583180,0.447278,0.493815,0.131532,0.279690,0.120202,...,0.024376,0.000000,0.072150,0.070676,0.129261,2.362122,2.386816,0.126262,0.147700,0.060564
2958,0.269162,0.115751,0.232515,0.191979,0.593526,0.454535,0.514874,0.167980,0.435275,0.155935,...,0.152948,0.062419,0.068437,0.024443,0.034861,2.322884,2.381585,0.255372,0.301815,0.091603
2959,0.451429,0.161668,0.233810,0.180955,0.629883,0.563681,0.746607,0.331831,0.314780,0.113117,...,0.319387,0.096774,0.061795,0.033857,0.090382,2.403871,2.418262,0.020127,0.050679,0.014684


In [14]:
pd.DataFrame(hog_features)

,40,41,42,43,44,45,46,47,48,49,...,126,127,128,129,130,131,132,133,134,135
0,0.185160,0.176298,0.219185,0.180706,0.295718,0.224165,0.295360,0.191292,0.104869,0.149954,...,0.179599,0.084671,0.115042,0.174627,0.231789,0.182001,0.165820,0.224235,0.216421,0.093830
1,0.251289,0.130244,0.230609,0.134020,0.254992,0.214456,0.311108,0.173736,0.120105,0.224234,...,0.215694,0.057203,0.070528,0.119666,0.260499,0.215123,0.142273,0.163087,0.221851,0.088803
2,0.170055,0.167817,0.183429,0.147178,0.228821,0.212980,0.208018,0.140653,0.097220,0.106405,...,0.192310,0.113940,0.147979,0.141443,0.155855,0.108190,0.248254,0.272212,0.282334,0.160843
3,0.172508,0.164443,0.190163,0.155892,0.246822,0.238708,0.211571,0.137734,0.105467,0.116038,...,0.132425,0.091886,0.225959,0.244018,0.127439,0.083753,0.375326,0.333211,0.146486,0.125690
4,0.165160,0.147439,0.177264,0.153569,0.233565,0.230609,0.201324,0.136322,0.096446,0.103678,...,0.112218,0.086347,0.294077,0.273480,0.082228,0.074803,0.412231,0.334227,0.113287,0.116490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,0.162669,0.117818,0.270699,0.286791,0.235364,0.172935,0.144744,0.096109,0.102723,0.101247,...,0.287969,0.154348,0.139607,0.206739,0.331150,0.156558,0.130390,0.195325,0.369846,0.194250
5642,0.311213,0.177306,0.209305,0.162084,0.270079,0.151703,0.108153,0.203880,0.264895,0.159976,...,0.237084,0.150464,0.201860,0.156991,0.275822,0.220752,0.180277,0.154248,0.315467,0.215578
5643,0.357724,0.177988,0.241843,0.289509,0.299873,0.139298,0.130931,0.286968,0.140271,0.134808,...,0.311059,0.201939,0.145539,0.164004,0.247635,0.167032,0.113965,0.147099,0.237550,0.146127
5644,0.360850,0.186802,0.189650,0.206035,0.308162,0.141522,0.148203,0.285650,0.215744,0.180990,...,0.282919,0.219240,0.118746,0.176129,0.174896,0.139455,0.122534,0.118368,0.129421,0.077471


In [15]:
cap = cv2.VideoCapture("./dataset/001-bg-01-090.avi")

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

for i in range(15):
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)        
    u = np.gradient(flow[:,0])
    v = np.gradient(flow[:,1])
    
    ux_grad = u[0]
    uy_grad = u[1]
    
    vx_grad = v[0]
    vy_grad = v[1]

cap.release()

In [16]:
pd.DataFrame(mbhx_features)

,244,245,246,247,248,249,250,251,252,253,...,330,331,332,333,334,335,336,337,338,339
0,0.125861,0.154881,0.149965,0.124884,0.125599,0.150298,0.143224,0.134865,0.153883,0.200960,...,0.181782,0.250452,0.223603,0.252137,0.245007,0.272384,0.203016,0.181028,0.126887,0.169541
1,0.110818,0.117980,0.110560,0.139374,0.120828,0.115037,0.104874,0.123104,0.185377,0.222131,...,0.229032,0.225930,0.123275,0.199808,0.242052,0.258297,0.191447,0.162530,0.123192,0.096202
2,0.191303,0.152721,0.112727,0.142619,0.175829,0.152410,0.116281,0.140594,0.203557,0.221966,...,0.156974,0.062121,0.223101,0.154490,0.125119,0.098964,0.223353,0.305343,0.280555,0.254020
3,0.178638,0.157034,0.160239,0.168871,0.181025,0.149158,0.137572,0.175709,0.236266,0.194458,...,0.180137,0.106500,0.206151,0.134984,0.154414,0.145041,0.259179,0.272808,0.188470,0.212506
4,0.164078,0.149465,0.153362,0.161061,0.168293,0.147348,0.154198,0.170954,0.192106,0.221251,...,0.113333,0.091550,0.180868,0.189134,0.185401,0.142934,0.157689,0.158840,0.134319,0.138646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,0.265326,0.358078,0.255258,0.350535,0.325677,0.145993,0.093797,0.146088,0.111220,0.187288,...,0.207149,0.206192,0.135095,0.173232,0.140224,0.119612,0.119645,0.166413,0.150063,0.136693
5642,0.145577,0.139586,0.146486,0.212885,0.226048,0.179599,0.142079,0.118448,0.257386,0.337479,...,0.097186,0.122783,0.162273,0.165190,0.164846,0.184966,0.152305,0.139592,0.141793,0.183205
5643,0.297856,0.385280,0.220809,0.232743,0.269324,0.204262,0.198690,0.205780,0.160981,0.204170,...,0.124924,0.185052,0.129887,0.138851,0.115637,0.119172,0.119217,0.162979,0.161015,0.152651
5644,0.372250,0.347977,0.201340,0.213337,0.230065,0.171756,0.158706,0.211747,0.190514,0.312391,...,0.090297,0.103897,0.128140,0.166092,0.144006,0.125077,0.126066,0.211683,0.137797,0.124206


# Implementation

### Separating files for training and testing

In [2]:
feature_files_path="./features/"
files=[os.path.join(feature_files_path,file) for file in os.listdir(feature_files_path)]

In [4]:
training_files = ["-nm-01-","-nm-02-","-nm-03-","-nm-04-"]
training_set_files = [file for file in files if any(training_file in file for training_file in training_files)]
training_set_files

['./features/1-nm-01-090.txt',
 './features/1-nm-02-090.txt',
 './features/1-nm-03-090.txt',
 './features/1-nm-04-090.txt',
 './features/10-nm-01-090.txt',
 './features/10-nm-02-090.txt',
 './features/10-nm-03-090.txt',
 './features/10-nm-04-090.txt',
 './features/100-nm-01-090.txt',
 './features/100-nm-02-090.txt',
 './features/100-nm-03-090.txt',
 './features/100-nm-04-090.txt',
 './features/101-nm-01-090.txt',
 './features/101-nm-02-090.txt',
 './features/101-nm-03-090.txt',
 './features/101-nm-04-090.txt',
 './features/102-nm-01-090.txt',
 './features/102-nm-02-090.txt',
 './features/102-nm-03-090.txt',
 './features/102-nm-04-090.txt',
 './features/103-nm-01-090.txt',
 './features/103-nm-02-090.txt',
 './features/103-nm-03-090.txt',
 './features/103-nm-04-090.txt',
 './features/104-nm-01-090.txt',
 './features/104-nm-02-090.txt',
 './features/104-nm-03-090.txt',
 './features/104-nm-04-090.txt',
 './features/105-nm-01-090.txt',
 './features/105-nm-02-090.txt',
 './features/105-nm-03

In [20]:
test_1_files = ["-nm-05-","-nm-06-"]
test_set_1_files = [file for file in files if any(training_file in file for training_file in test_1_files)]
test_set_1_files

['./features/1-nm-05-090.txt',
 './features/1-nm-06-090.txt',
 './features/10-nm-05-090.txt',
 './features/10-nm-06-090.txt',
 './features/100-nm-05-090.txt',
 './features/100-nm-06-090.txt',
 './features/101-nm-05-090.txt',
 './features/101-nm-06-090.txt',
 './features/102-nm-05-090.txt',
 './features/102-nm-06-090.txt',
 './features/103-nm-05-090.txt',
 './features/103-nm-06-090.txt',
 './features/104-nm-05-090.txt',
 './features/104-nm-06-090.txt',
 './features/105-nm-05-090.txt',
 './features/105-nm-06-090.txt',
 './features/106-nm-05-090.txt',
 './features/106-nm-06-090.txt',
 './features/107-nm-05-090.txt',
 './features/107-nm-06-090.txt',
 './features/108-nm-05-090.txt',
 './features/108-nm-06-090.txt',
 './features/109-nm-05-090.txt',
 './features/109-nm-06-090.txt',
 './features/11-nm-05-090.txt',
 './features/11-nm-06-090.txt',
 './features/110-nm-05-090.txt',
 './features/110-nm-06-090.txt',
 './features/111-nm-05-090.txt',
 './features/111-nm-06-090.txt',
 './features/112-n

In [21]:
test_2_files = ["-bg-01-","-bg-02-"]
test_set_2_files = [file for file in files if any(training_file in file for training_file in test_2_files)]
test_set_2_files

['./features/1-bg-01-090.txt',
 './features/1-bg-02-090.txt',
 './features/10-bg-01-090.txt',
 './features/10-bg-02-090.txt',
 './features/100-bg-01-090.txt',
 './features/100-bg-02-090.txt',
 './features/101-bg-01-090.txt',
 './features/101-bg-02-090.txt',
 './features/102-bg-01-090.txt',
 './features/102-bg-02-090.txt',
 './features/103-bg-01-090.txt',
 './features/103-bg-02-090.txt',
 './features/104-bg-01-090.txt',
 './features/104-bg-02-090.txt',
 './features/105-bg-01-090.txt',
 './features/105-bg-02-090.txt',
 './features/106-bg-01-090.txt',
 './features/106-bg-02-090.txt',
 './features/107-bg-01-090.txt',
 './features/107-bg-02-090.txt',
 './features/108-bg-01-090.txt',
 './features/108-bg-02-090.txt',
 './features/109-bg-01-090.txt',
 './features/109-bg-02-090.txt',
 './features/11-bg-01-090.txt',
 './features/11-bg-02-090.txt',
 './features/110-bg-01-090.txt',
 './features/110-bg-02-090.txt',
 './features/111-bg-01-090.txt',
 './features/111-bg-02-090.txt',
 './features/112-b

In [22]:
test_3_files = ["-cl-01-","-cl-02-"]
test_set_3_files = [file for file in files if any(training_file in file for training_file in test_3_files)]
test_set_3_files

['./features/1-cl-01-090.txt',
 './features/1-cl-02-090.txt',
 './features/10-cl-01-090.txt',
 './features/10-cl-02-090.txt',
 './features/100-cl-01-090.txt',
 './features/100-cl-02-090.txt',
 './features/101-cl-01-090.txt',
 './features/101-cl-02-090.txt',
 './features/102-cl-01-090.txt',
 './features/102-cl-02-090.txt',
 './features/103-cl-01-090.txt',
 './features/103-cl-02-090.txt',
 './features/104-cl-01-090.txt',
 './features/104-cl-02-090.txt',
 './features/105-cl-01-090.txt',
 './features/105-cl-02-090.txt',
 './features/106-cl-01-090.txt',
 './features/106-cl-02-090.txt',
 './features/107-cl-01-090.txt',
 './features/107-cl-02-090.txt',
 './features/108-cl-01-090.txt',
 './features/108-cl-02-090.txt',
 './features/109-cl-01-090.txt',
 './features/109-cl-02-090.txt',
 './features/11-cl-01-090.txt',
 './features/11-cl-02-090.txt',
 './features/110-cl-01-090.txt',
 './features/110-cl-02-090.txt',
 './features/111-cl-01-090.txt',
 './features/111-cl-02-090.txt',
 './features/112-c

## Codebook for training data

In [102]:
hog = pd.DataFrame()
mbhx = pd.DataFrame()
mbhy = pd.DataFrame()
for file in files:
    
    features = pd.read_csv(file,'\t',header=None)
    features.drop(features.columns[-1],axis=1,inplace=True)
    
    hog = hog.append(features.iloc[:,trajectory_range:hog_range], ignore_index=True)
    mbhx = mbhx.append(features.iloc[:,hof_range:mbhx_range], ignore_index=True)
    mbhy = mbhy.append(features.iloc[:,mbhx_range:mbhy_range], ignore_index=True)

In [8]:
images_neg = []
hog = []
mbhx = []
mbhy = []
for file in training_set_files:
    
    features = pd.read_csv(file,'\t',header=None)
    features.drop(features.columns[-1],axis=1,inplace=True)
    
    hog.append(features.iloc[:,trajectory_range:hog_range])
    mbhx.append(features.iloc[:,hof_range:mbhx_range])
    mbhy.append(features.iloc[:,mbhx_range:mbhy_range])

In [9]:
print(len(hog))
print(len(mbhx))
print(len(mbhy))

496
496
496


In [10]:
hog_df = pd.DataFrame()
mbhx_df = pd.DataFrame()
mbhy_df = pd.DataFrame()

for i in range(len(hog)):    
    hog_df = hog_df.append(hog[i])
    
for i in range(len(mbhx)):    
    mbhx_df = mbhx_df.append(mbhx[i])
    
for i in range(len(mbhy)):    
    mbhy_df = mbhy_df.append(mbhy[i])

In [15]:
print(hog_df.shape)
print(mbhx_df.shape)
print(mbhy_df.shape)

(2969163, 96)
(2969163, 96)
(2969163, 96)


In [16]:
hog_df.to_csv('hog_features_training.csv')
mbhx_df.to_csv('mbhx_features_training.csv')
mbhy_df.to_csv('mbhy_features_training.csv')

In [18]:
hog_df.reset_index()

,index,40,41,42,43,44,45,46,47,48,...,126,127,128,129,130,131,132,133,134,135
0,0,0.096640,0.104327,0.191246,0.142583,0.150943,0.271287,0.363926,0.150873,0.150382,...,0.236547,0.142430,0.161037,0.195968,0.242291,0.264078,0.177749,0.247496,0.183371,0.088757
1,1,0.179560,0.168156,0.182668,0.160355,0.247472,0.210503,0.213418,0.139407,0.104483,...,0.120828,0.090646,0.210091,0.242323,0.138184,0.093227,0.335822,0.315821,0.172895,0.128468
2,2,0.104454,0.063144,0.092169,0.107886,0.194134,0.147594,0.134969,0.081215,0.109249,...,0.157648,0.085201,0.206093,0.236002,0.174563,0.109853,0.347193,0.363824,0.215805,0.145705
3,3,0.097249,0.054644,0.085554,0.121885,0.217539,0.149219,0.119695,0.065027,0.106549,...,0.157240,0.096439,0.200269,0.225659,0.143116,0.090160,0.312607,0.370462,0.252419,0.149960
4,4,0.107119,0.044974,0.087704,0.067964,0.209612,0.156017,0.152299,0.075884,0.214531,...,0.145742,0.092034,0.182288,0.201485,0.127175,0.102141,0.383946,0.399268,0.135519,0.101435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2969158,5785,0.164543,0.177467,0.133931,0.188438,0.203009,0.194467,0.191857,0.121537,0.089001,...,0.219720,0.138812,0.124489,0.187857,0.334618,0.161945,0.126191,0.166803,0.348143,0.181339
2969159,5786,0.213009,0.159401,0.096544,0.165099,0.203133,0.164150,0.155981,0.204000,0.361611,...,0.192456,0.114473,0.272548,0.158807,0.234573,0.144320,0.252426,0.129745,0.211497,0.137117
2969160,5787,0.253645,0.191839,0.110237,0.239067,0.308578,0.213806,0.176130,0.132667,0.104143,...,0.247060,0.174942,0.123438,0.146701,0.218241,0.147851,0.100352,0.131729,0.225951,0.130684
2969161,5788,0.219414,0.157490,0.101928,0.134990,0.236939,0.184583,0.129554,0.207324,0.357041,...,0.173249,0.131371,0.237046,0.169149,0.258016,0.150848,0.182103,0.134491,0.218629,0.156256


In [58]:
hog.shape

(1240,)